# Module 4

Score rules. Please solve exercises worth at most 30 pts. For each correct execution you will receive an assigned number of points. Its sum will determine the final score.


1. Implement Diffie-Hellman key exchange on an ellptic curve (missing: 0 pts, implementation up to 10 pts)
2. Implement a simple block cipher (missing: 0 pts, implementation up to 10 pts)
3. Design a hybrid system (missing: 0 pts, implementation up to 10 pts)

# 1. Implement Diffie-Hellman key exchange on an ellptic curve
Input: 
1. An elliptic curve $E:y^2=x^3+Ax+B$ over a finite field $\mathbb{F}_{p}$, $p$-prime number.
2. A point $P$ of order $q$ in $E(\mathbb{F}_{p})$.

Output: A shared secret point $Q$ common to Alice and Bob which is $Q=kP$, $k$ is secretely generated in the followin process:

Alice generates a random multiple $P_{A}=xP$ of the point $P$, $0<x<q$

Bob generates a random multiple $P_{B}=yP$ of the point $P$, $0<y<q$.

They exchange the points $P_{A}$ and $P_{B}$ and generate the multiple $Q=kP$, where $k=xy$.

Emulate such an exchange between Alice and Bob.

Generate a sequence of $128$ bits out of the point $Q$. This will be the key used in algorithm $3$.

In [109]:
def zero(p):
    return p[0] == 0  and p[1] == 0;

def order(EC, P):
    for i in range(1, EC[2] + 1):
        if zero(MultPoint(EC, P, i)):
            return i
        pass
    raise Exception("Invalid order")

def ExtendedGCD(a,b):
    r,r1=a,b
    s,s1=1,0 #s*a+t*b == a
    t,t1=0,1 #s1*a+t1*b == b
    while not(r1==0):
        q,r2=r//r1,r % r1
        r,s,t,r1,s1,t1=r1,s1,t1,r2,s-s1*q,t-t1*q
    d=r
    return d,s,t #s*a+t*b=d, d=GCD(a,b)

def inv(a,p):
    d,i,_ = ExtendedGCD(a,p);
    if(d!=1):
        raise Exception("The number a and p are not coprime")
    return i%p;

#Dividing is the same as multiplying by the inv
def PtSum(A,B,p,x1,y1,x2,y2):
    def zero(x,y):
        return x == 0  and y == 0;
    if(zero(x1,y1)):
        return [x2,y2]
    if(zero(x2,y2)):
        return [x1,y1]
    if x1 == x2 and (y1 != y2 or y1 == 0):
        return [0,0] #p1 + (-p1) 
    if(x1 == x2 and y1 == y2):
        resX = ((3*x1**2 + A) * inv(2*y1,p))%p
        x3 = ((resX * resX) - 2 * x1) % p
        y3 = (resX * (x1-x3) - y1) %p
        return [x3,y3]
    else:
        resX = ((y2-y1) * inv(x2-x1,p))%p
        x3 =  (resX * resX - x1 -x2)%p
        y3 = ((y2 -y1) * inv(x2-x1,p)*(x1-x3)-y1)%p
        return [x3,y3]
    
def OppPt(x,y,p):
    return [x, (-y % p)]
#Random k-bit number generator
def gennum(size):
    li=[random.choice([0,1]) for k in range(0,size-2)]
    li.append(1)
    num=li[0]
    for el in li[1:]:
        num<<=1
        num^=el
    return num

def gcd(a,b):
    while b:
        a,b = b,a % b;
    return abs(a);

def get_coprime(n):
    while True:
        coprime = random.randrange(2,n)
        if gcd(coprime, n) == 1:
            return coprime
        else:
            return -1
        
#Returns true if prime after count rounds 
def FermatTest(n,count=100000):
    if n == 1 or n == 2: return True;
    for i in range(count):
        a = get_coprime(n)
        if(a < 0 or pow(a,n-1,n) != 1):
            return False
    return True;

def PrimeGen(size):
    num=gennum(size)
    while not FermatTest(num,size):
        num=gennum(size)
    return num

def PrimeGenMod4(size):
    if(size < 4): raise Exception("Size should be >= 4");
    num = PrimeGen(size);
    while (num <= 3) or (num%4 != 3):
        num = PrimeGen(size);
    return num

def RandEllCurve(k):
    p = PrimeGenMod4(k);
    A=random.randint(0,p-1)
    B=random.randint(0,p-1)
    while (((4*(A**3) + 27*(B**2))%p) == 0):
        A=random.randint(0,p-1)
        B=random.randint(0,p-1)
    return [A , B , p]

def zero(p):
    return p[0] == 0  and p[1] == 0;

def order(EC, P):
    for i in range(1, EC[2] + 1):
        if zero(MultPoint(EC, P, i)):
            return i
        pass
    raise Exception("Invalid order")


#Square root modulo
#We assume that p is prime since condition 1 has to be met
def sqrt(a, p):

    def legendre_symbol(a, p):
        ls = pow(a, (p - 1) // 2, p)
        return -1 if ls == p - 1 else ls

    if legendre_symbol(a, p) != 1:
        return -1, -1
    elif a == 0:
        return -1- -1
    elif p == 2:
        return p
    elif p % 4 == 3:
        result = pow(a, (p + 1) // 4, p)
        return result, abs(p-result)

    s = p - 1
    e = 0
    while s % 2 == 0:
        s //= 2
        e += 1
    n = 2
    while legendre_symbol(n, p) != -1:
        n += 1
    x = pow(a, (s + 1) // 2, p)
    b = pow(a, s, p)
    g = pow(n, s, p)
    r = e

    while True:
        t = b
        m = 0
        for m in range(r):
            if t == 1:
                break
            t = pow(t, 2, p)

        if m == 0:
            return x,abs(p-x)

        gs = pow(g, 2 ** (r - m - 1), p)
        g = (gs * gs) % p
        x = (x * gs) % p
        b = (b * g) % p
        r = m

def RandomPointOrder(A,B,p,k):
    cond = A < p-1 and B < p-1 and A > 0 and B > 0;
    #Natural number p which is prime and satisfies the condition p mod 4 = 3.
    if(cond and (p%4!=3) and not FermatTest(p)):
        raise Exception("Conditions are not met")
    order =  k.bit_length()//4
    x = random.randint(1,p-1)
    sq = (x ** 3 + A * x + B) % p
    y1,y2 = sqrt(sq, p)
    while y1 == -1 and orderPoint([A,B,p],[x,y1]) > order:
        x = random.randrange(0,p-1)
        sq = (x ** 3 + A * x + B) % p
        y1,y2 = sqrt(sq, p)
    y = random.choice([y1,y2])
    return [x,y]

In [110]:
def generateSequence(Q):
    return (hash(Q) % 2**128)

def AliceToBob(E,P):
    q = order(E, P)
    x = random.randint(1, q-1)
    Pa = MultPoint(E,P,x)
    print("Alice public key " + Pa[0] + Pa[1])
    return (Pa,x)

def BobToAlice(E,P):
    q = order(E, P)
    y = random.randint(1, q-1)
    Pb = MultPoint(E,P,y)
    print("Bob public key " + Pb[0] + Pb[1])
    return (Pb,y)
def DiffieHellmanExchange(E,P):
    #emulate this action with prints and return an actual point on the selected curve
    Alice = AliceToBob(E,P)
    alicePrivate = Alice[1]
    Bob = BobToAlice(E,P)
    bobPrivate = Bob[1]
    aliceShared = MultPoint(E,P,alicePrivate)
    print("alice shared key: (",aliceShared[0],", ", aliceShared[1],")")
        
    BobShared = MultPoint(E,P,bobPrivate)
    print("bob shared key: (",BobShared[0],", ", BobShared[1],")")
    print("Bit sequence Q: \n" + bin(generateSequence(BobShared[0]))[2:].zfill(128))

# 2. Implement a simple block cipher

Design a simple block cipher which operates on a block of $32$ bits, called $B$ and in each round of the cipher performs the following actions:

A) SBox(B): SBox function can be any reversible function which transforms the block $B$ into another block $B'$ with 32 blocks.

B) RoundXOR(B,K): Applying XOR function to the block $B$ with the round key $K$.

C) Perm(B): Apply a permutation function which swaps the bits in the block $B$ in the following way: divide the block into four blocks of $8$ bits and then label the subblocks as $(1,2,3,4)$. Produce a new block $B'$ which has the structure $(3,1,4,2)$

Now your cipher should operate on the input block $B$ as follows: 

1. Produce round keys $K_1, K_2, K_3, K_4$ of size $32$ bits from the $128$ bit key generated in the algorithm

In [113]:
#Gen Params
SIZE = 40
p,q = GenTwoPrimes(SIZE);
phi = (p-1)*(q-1);
e = RandomE(phi);
d = inv(e,phi)
#Random k-bit number generator
def gennum(size):
    li=[random.choice([0,1]) for k in range(0,size-2)]
    li.append(1)
    num=li[0]
    for el in li[1:]:
        num<<=1
        num^=el
    return num

def get_coprime(n):
    while True:
        coprime = random.randrange(2,n)
        if gcd(coprime, n) == 1:
            return coprime
        else:
            return -1
        
#Returns true if prime after count rounds 
def FermatTest(n,count=100000):
    if n == 1 or n == 2: return True;
    for i in range(count):
        a = get_coprime(n)
        if(a < 0 or pow(a,n-1,n) != 1):
            return False
    return True;

def PrimeGen(size):
    num=gennum(size)
    while not FermatTest(num,size):
        num=gennum(size)
    return num

def GenTwoPrimes(size):
    while True:
        p=PrimeGen(size)
        q=PrimeGen(size)
        if p!=q:
            return p,q
        
def GCD(a,b):
    while b:   
        a, b = b, a % b
    return abs(a)

def RandomE(phin):
    size=phin.bit_length()
    sizen=random.randrange(2,size)
    while True:
        num=gennum(sizen)
        if GCD(num,phin)==1 and num>1:
            return num

def ProduceRoundKey(inpK):
    return [inpK[0:32],inpK[32:64],inpK[64:96],inpK[96:128]]


    
def RoundXOR(a, b): 
    ans = "" 
    for i in range(len(a)): 
        if a[i] == b[i]: 
            ans = ans + "0"
        else: 
            ans = ans + "1"
    return ans    

def SBox(state):
    num = int("".join(str(x) for x in state), 2)
    num = num ** e;
    res = [int(x) for x in bin(num)[2:]]
    return res;

def InvSBox(state):
    num = int("".join(str(x) for x in state), 2)
    num = num **d;
    res = [int(x) for x in bin(num)[2:]]
    return res;

def Permutation(state):
    #(3,1,4,2)
    list1 = list(state)
    list2 = [aux1[0:8], aux1[8,16], aux1[16, 24], aux1[24:32]]
    perm = [aux2[2], aux2[0], aux2[3], aux2[1]]
    return perm
    pass

def InvPerm(state):
    aux1 = list(state)
    aux2 = [aux1[0:8], aux1[8,16], aux1[16, 24], aux1[24:32]]
    invPerm = [aux2[1], aux2[3], aux2[0], aux2[2]]
    return invPerm
    pass

2. Coding procedure

In [114]:
def Encode(B,K): #K is a list of four 32 bit keys produced from the 128 bit key from alg. 1
    state=B
    for i in [0,1,2]:
        state=RoundXOR(state,K[i])
        state=SBox(state)
        state=Perm(state)

    state=RoundXOR(state,K[3])
    return state

2. Decoding procedure

Implement `InvPerm`, `InvSBox` - inverses of the appropriate functions.

In [115]:
def Decode(C,K):
    state=C
    state=RoundXOR(state,K[4])

    for i in [3,2,1]:
        state=InvPerm(state)
        state=InvSBox(state)
        state=RoundXOR(state,K[i])
    return state

In [ ]:
keynum = 324234;
messageN
key = [int(x) for x in bin(keynum)[2:]]
m = [int(x) for x in bin(keynum)[2:]]
encode()

# 3. Design a hybrid system

Design a system in which the two parties Alice and Bob generate a common $128$ bit key `inpK` using procedure 1. Next, they establish a communication protocol in which they send blocks of $32$ bit size which are encoded using the procedure `Encode(B,K)` where `K` is a list of four $32$ bit blocks produced from `inpK`. 

Alice is encoding a message to Bob with `C=Encode(B,K)` and Bob decrypts it using `B=Decode(C,K)`. 

Emulate this system with prints and show how the message is being transferred.

You can allow the user to send text on the input, process this plaintext into chunks of size $32$ bits, encrypt it, send it, decode it and reconstruct the plaintext from the decoded chunks (design your own procedure for that).



In [ ]:
E = RandEllCurve(30)
DiffieHellmanExchange(E,P)